In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('data.csv')
filtered_df = pd.read_csv('filtered_data.csv')

In [ ]:
df = df[df['Year'].between(2018, 2022)]
df = df[df['Year'] != 2020]
df.info()

In [ ]:
# Convert 'Hx_GDM' and 'Other Endocrine probs' to object type
df['Hx_GDM'] = df['Hx_GDM'].astype('object')
df['Other Endocrine probs'] = df['Other Endocrine probs'].astype('object')

# Verify the changes
print(df.dtypes[['Hx_GDM', 'Other Endocrine probs']])

In [ ]:
# Merge dataframes on relevant key (assuming there's an 'ID' column or similar to merge on)
merged_df = df.merge(filtered_df[['ID', 'Date of Birth','GDM Val', 'matched']], on=['ID', 'Date of Birth'], how='left')

# Update GDM labels
merged_df.loc[(merged_df['GDM Val'] == 1) & (merged_df['GDM'] == 0), 'GDM'] = 1
merged_df = merged_df[~((merged_df['GDM'] == 1) & (merged_df['GDM Val'] == 0))]

# Drop unnecessary columns
merged_df.drop(columns=['GDM Val', 'matched'], inplace=True)
merged_df = merged_df.drop(['Date of Birth', 'Endocrine probs.', 'Year', 'Height of Mother', 'Ever Smoked_x', 'Epid/Spinal Complications',
              'Onset of Labour', 'Multiple Pregnancy','Blood Loss', 'Maternal blood taken','Maternal investigations','Post Delivery Problems',
               'Perineum/other tears', 'Occupation - Mother', 'CTG/ECG anomalies', 'Shoulder Difficulty', 'Weight of Baby', 'Sex of Baby',
               'Apgar at 1 minute', 'Apgar at 5 minutes', 'Outcome', 'Resuscitation', 'Transferred to', 'ISCO Title', 'Medical probs in Preg', 'Drugs this pregnancy', 'ISCO Group'], axis=1)

merged_df = merged_df.drop_duplicates()

In [ ]:
#Subset the data for training
df = df.drop(['Date of Birth', 'Year', 'Endocrine probs.', 'Height of Mother', 'Ever Smoked_x', 'Epid/Spinal Complications',
              'Onset of Labour', 'Multiple Pregnancy','Blood Loss', 'Maternal blood taken','Maternal investigations','Post Delivery Problems',
               'Perineum/other tears', 'Occupation - Mother', 'CTG/ECG anomalies', 'Shoulder Difficulty', 'Weight of Baby', 'Sex of Baby',
               'Apgar at 1 minute', 'Apgar at 5 minutes', 'Outcome', 'Resuscitation', 'Transferred to', 'ISCO Title', 'Medical probs in Preg', 'Drugs this pregnancy', 'ISCO Group'], axis=1)
df = df.drop_duplicates()

# Train on Raw

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve, precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve
from sklearn.linear_model import LogisticRegression as SKLogit
from sklearn.calibration import calibration_curve
from sklearn.linear_model import LinearRegression
from scipy.special              import logit, expit


# Combine the datasets and create a source column
df['source'] = 'raw'
merged_df['source'] = 'validated'
combined_df = pd.concat([df, merged_df], ignore_index=True)

# Define categorical and numerical features
categorical_features = combined_df.select_dtypes(include=['object']).columns.tolist()
numerical_features = combined_df.select_dtypes(exclude=['object']).columns.tolist()
categorical_features.remove('source')
numerical_features.remove('GDM')
numerical_features.remove('ID')

# Preprocessor setup
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), categorical_features),
        ('num', StandardScaler(), numerical_features)
    ],
    remainder='passthrough'
)

# Apply preprocessing to the entire dataset except 'GDM' and 'ID'
X = combined_df.drop(columns=['GDM', 'ID'])
X_processed = preprocessor.fit_transform(X)
y = combined_df['GDM'].values

# Convert processed features back to DataFrame with transformed column names
columns_transformed = (list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features))
                       + numerical_features + ['source'])
X_processed_df = pd.DataFrame(X_processed, columns=columns_transformed)
X_processed_df['ID'] = combined_df['ID'].values
X_processed_df['source'] = combined_df['source'].values

# Perform group-based split to create train and test sets
gss = GroupShuffleSplit(n_splits=1, train_size=0.7, random_state=42)
train_idxs, test_idxs = next(gss.split(X_processed_df, groups=X_processed_df['ID']))

train_set = X_processed_df.iloc[train_idxs]
test_set = X_processed_df.iloc[test_idxs]

# Now subset the train and test sets based on the 'source' column
# Train on raw data
raw_mask = train_set['source'] == 'raw'
X_train_raw = train_set[raw_mask].drop(columns=['ID', 'source'])
y_train_raw = y[train_set[raw_mask].index]

# Test on validated data
validated_mask = test_set['source'] == 'validated'
X_test_validated = test_set[validated_mask].drop(columns=['ID', 'source'])
y_test_validated = y[test_set[validated_mask].index]

# Initialize the logistic regression model
logistic_regression = LogisticRegression(max_iter=1000, random_state=42)

# Perform k-fold cross-validation on raw training data
from sklearn.model_selection import StratifiedKFold, cross_val_score
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
roc_auc_scores = cross_val_score(logistic_regression, X_train_raw, y_train_raw, cv=kfold, scoring='roc_auc')
avg_precision_scores = cross_val_score(logistic_regression, X_train_raw, y_train_raw, cv=kfold, scoring='average_precision')

# Train the model on the entire raw training set
logistic_regression.fit(X_train_raw, y_train_raw)

# Predict on the validated test set
y_pred_prob_validated = logistic_regression.predict_proba(X_test_validated)[:, 1]

# Evaluate the model
roc_auc_validated = roc_auc_score(y_test_validated, y_pred_prob_validated)
avg_precision_validated = average_precision_score(y_test_validated, y_pred_prob_validated)

print(f"ROC AUC (Validated Data): {roc_auc_validated:.3f}")
print(f"Average Precision (Validated Data): {avg_precision_validated:.3f}")
print(f"Cross-validated ROC AUC scores (Raw Data): {roc_auc_scores}")
print(f"Cross-validated Average Precision scores (Raw Data): {avg_precision_scores}")

# ─── Calibration curve (non-parametric bins) ────────────────────────────────
prob_true, prob_pred = calibration_curve(
    y_test_validated,
    y_pred_prob_validated,
    n_bins=10,
    strategy='uniform'
)

# linear least-squares in prob space
lr = LinearRegression().fit(prob_pred.reshape(-1,1), prob_true)
b0, b1 = lr.intercept_, lr.coef_[0]
print(f"Linear calib intercept: {b0:.3f}")
print(f"Linear calib slope:     {b1:.3f}")

# 5b) fit logistic in log-odds space
calib = LogisticRegression(penalty=None, solver='lbfgs', max_iter=1000)
eps   = 1e-15
p_clip = np.clip(y_pred_prob_validated, eps, 1-eps)
logodds = logit(p_clip).reshape(-1,1)
calib.fit(logodds, y_test_validated)

α = calib.intercept_[0]
β = calib.coef_[0,0]
print(f"Calibration intercept (logit scale): {α:.3f}")
print(f"Calibration slope     (logit scale): {β:.3f}")

# O : E ratio
exp_pos = np.sum(y_pred_prob_validated)
obs_pos = np.sum(y_test_validated)
oe_ratio = obs_pos/exp_pos if exp_pos>0 else np.nan
print(f"O:E ratio (obs/exp): {oe_ratio:.3f}")


# ─── Plot all three panels: ROC, PR, and new Calibration ────────────────────
fpr, tpr, _        = roc_curve(y_test_validated, y_pred_prob_validated)
precision, recall, _ = precision_recall_curve(y_test_validated, y_pred_prob_validated)

plt.figure(figsize=(15, 4))

# ROC
plt.subplot(1, 3, 1)
plt.plot(fpr_rf, tpr_rf, label=f'ROC AUC = {roc_auc_validated:.3f}')
plt.plot([0,1],[0,1],'k--', lw=0.8)
plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
plt.title('ROC Curve'); plt.legend(loc='lower right')

# Precision-Recall
plt.subplot(1, 3, 2)
plt.plot(recall_rf, precision_rf, label=f'AP = {avg_precision_validated:.3f}')
plt.xlabel('Recall'); plt.ylabel('Precision')
plt.title('Precision–Recall Curve'); plt.legend(loc='lower left')

# Calibration
plt.subplot(1, 3, 3)
plt.plot(prob_pred, prob_true, marker='o', linestyle='-',
         label='Calibration')
plt.plot([0, 1], [0, 1], 'k--', lw=0.8, label='Ideal')

plt.xlabel('Predicted probability')
plt.ylabel('Observed proportion')
plt.title('Calibration curve')
plt.text(0.05, 0.85,
         f'Intercept = {α:.2f}\nSlope = {β:.2f}',
         transform=plt.gca().transAxes,
         bbox=dict(boxstyle='round', alpha=0.1))

plt.legend(loc='lower right')
plt.tight_layout()
plt.show()

In [ ]:
val_gdm_true = y_test_validated
ehr_gdm_prob = y_pred_prob_validated

In [ ]:

plt.figure(figsize=(13, 7), dpi=300)

plt.subplot(1, 2, 1)
plt.plot(fpr, tpr, label=f'ROC AUC = {roc_auc_validated:.3f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=24)
plt.ylabel('True Positive Rate', fontsize=24)
plt.title('ROC Curve', fontsize=28)
plt.legend(loc="lower right", fontsize=20)

plt.subplot(1, 2, 2)
plt.plot(recall, precision, label=f'Average Precision = {avg_precision_validated:.3f}')
plt.xlabel('Recall', fontsize=24)
plt.ylabel('Precision', fontsize=24)
plt.title('Precision-Recall Curve', fontsize=28)
plt.legend(loc="lower left", fontsize=20)

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(13, 7), dpi=300)
plt.plot(prob_pred, prob_true, marker='o', linestyle='-', label='Calibration')
plt.plot([0, 1], [0, 1], 'k--', lw=0.8, label='Ideal')
plt.xlabel('Predicted probability'); plt.ylabel('Observed proportion')
plt.title('Calibration curve')
plt.text(0.05, 0.85,
         f'Intercept = {α:.2f}\nSlope = {β:.2f}',
         transform=plt.gca().transAxes,
         bbox=dict(boxstyle='round', alpha=0.1))
plt.legend(loc='lower right')

## RF Raw

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the Random Forest model
random_forest = RandomForestClassifier(random_state=42)

# Perform k-fold cross-validation
roc_auc_scores_rf = cross_val_score(random_forest, X_train_raw, y_train_raw, cv=kfold, scoring='roc_auc')
avg_precision_scores_rf = cross_val_score(random_forest, X_train_raw, y_train_raw, cv=kfold, scoring='average_precision')

# Train the model on the entire training set
random_forest.fit(X_train_raw, y_train_raw)

# Predict on the test set
y_pred_prob_rf = random_forest.predict_proba(X_test_validated)[:, 1]

# Evaluate the model
roc_auc_rf = roc_auc_score(y_test_validated, y_pred_prob_rf)
avg_precision_rf = average_precision_score(y_test_validated, y_pred_prob_rf)

print(f"ROC AUC (Random Forest): {roc_auc_rf:.3f}")
print(f"Average Precision (Random Forest): {avg_precision_rf:.3f}")
print(f"Cross-validated ROC AUC scores (Random Forest): {roc_auc_scores_rf}")
print(f"Cross-validated Average Precision scores (Random Forest): {avg_precision_scores_rf}")


# ─── Calibration curve (non-parametric bins) ────────────────────────────────
prob_true, prob_pred = calibration_curve(
    y_test_validated,
    y_pred_prob_rf,
    n_bins=10,
    strategy='uniform'
)

# linear least-squares in prob space
lr = LinearRegression().fit(prob_pred.reshape(-1,1), prob_true)
b0, b1 = lr.intercept_, lr.coef_[0]
print(f"Linear calib intercept: {b0:.3f}")
print(f"Linear calib slope:     {b1:.3f}")

# 5b) fit logistic in log-odds space
calib = LogisticRegression(penalty=None, solver='lbfgs', max_iter=1000)
eps   = 1e-15
p_clip = np.clip(y_pred_prob_rf, eps, 1-eps)
logodds = logit(p_clip).reshape(-1,1)
calib.fit(logodds, y_test_validated)

α = calib.intercept_[0]
β = calib.coef_[0,0]
print(f"Calibration intercept (logit scale): {α:.3f}")
print(f"Calibration slope     (logit scale): {β:.3f}")

# O : E ratio
exp_pos = np.sum(y_pred_prob_rf)
obs_pos = np.sum(y_test_validated)
oe_ratio = obs_pos/exp_pos if exp_pos>0 else np.nan
print(f"O:E ratio (obs/exp): {oe_ratio:.3f}")

# Plot ROC and Precision-Recall curves
fpr_rf, tpr_rf, _ = roc_curve(y_test_validated, y_pred_prob_rf)
precision_rf, recall_rf, _ = precision_recall_curve(y_test_validated, y_pred_prob_rf)

plt.figure(figsize=(10, 5))

plt.subplot(1, 3, 1)
plt.plot(fpr_rf, tpr_rf, label=f'ROC AUC = {roc_auc_rf:.3f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")

plt.subplot(1, 3, 2)
plt.plot(recall_rf, precision_rf, label=f'Average Precision = {avg_precision_rf:.3f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")

# Calibration
plt.subplot(1, 3, 3)
plt.plot(prob_pred, prob_true, marker='o', linestyle='-',
         label='Calibration')
plt.plot([0, 1], [0, 1], 'k--', lw=0.8, label='Ideal')

plt.xlabel('Predicted probability')
plt.ylabel('Observed proportion')
plt.title('Calibration curve')
plt.text(0.05, 0.85,
         f'Intercept = {α:.2f}\nSlope = {β:.2f}',
         transform=plt.gca().transAxes,
         bbox=dict(boxstyle='round', alpha=0.1))
plt.legend(loc='lower right')

plt.tight_layout()
plt.show()

## XGB Raw

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve, precision_recall_curve
import matplotlib.pyplot as plt

xgboost = XGBClassifier(eval_metric='logloss', random_state=42)

X_train_raw      = X_train_raw.astype(np.float32)
X_test_validated = X_test_validated.astype(np.float32)

# Perform k-fold cross-validation
roc_auc_scores_xgb = cross_val_score(xgboost, X_train_raw, y_train_raw, cv=kfold, scoring='roc_auc')
avg_precision_scores_xgb = cross_val_score(xgboost, X_train_raw, y_train_raw, cv=kfold, scoring='average_precision')

# Train the model on the entire training set
xgboost.fit(X_train_raw, y_train_raw)

# Predict on the test set
y_pred_prob_xgb = xgboost.predict_proba(X_test_validated)[:, 1]

# Evaluate the model
roc_auc_xgb = roc_auc_score(y_test_validated, y_pred_prob_xgb)
avg_precision_xgb = average_precision_score(y_test_validated, y_pred_prob_xgb)

print(f"ROC AUC (XGBoost): {roc_auc_xgb:.3f}")
print(f"Average Precision (XGBoost): {avg_precision_xgb:.3f}")
print(f"Cross-validated ROC AUC scores (XGBoost): {roc_auc_scores_xgb}")
print(f"Cross-validated Average Precision scores (XGBoost): {avg_precision_scores_xgb}")

# Plot ROC and Precision-Recall curves
fpr_xgb, tpr_xgb, _ = roc_curve(y_test_validated, y_pred_prob_xgb)
precision_xgb, recall_xgb, _ = precision_recall_curve(y_test_validated, y_pred_prob_xgb)

# ─── Calibration curve (non-parametric bins) ────────────────────────────────
prob_true, prob_pred = calibration_curve(
    y_test_validated,
    y_pred_prob_xgb,
    n_bins=10,
    strategy='uniform'
)


# 5b) fit logistic in log-odds space
calib = LogisticRegression(penalty=None, solver='lbfgs', max_iter=1000)
eps   = 1e-15
p_clip = np.clip(y_pred_prob_xgb, eps, 1-eps)
logit_y_score = np.log(p_clip / (1 - p_clip))
calib.fit(logit_y_score.reshape(-1, 1), y_test_validated)
calib_intercept = calib.intercept_[0]
calib_slope = calib.coef_[0][0]

print(f"Calibration intercept (logit scale): {calib_intercept:.3f}")
print(f"Calibration slope     (logit scale): {calib_slope:.3f}")

# O : E ratio
exp_pos = np.sum(y_pred_prob_xgb)
obs_pos = np.sum(y_test_validated)
oe_ratio = obs_pos/exp_pos if exp_pos>0 else np.nan
print(f"O:E ratio (obs/exp): {oe_ratio:.3f}")


plt.figure(figsize=(10, 5))

plt.subplot(1, 3, 1)
plt.plot(fpr_xgb, tpr_xgb, label=f'ROC AUC = {roc_auc_xgb:.3f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")

plt.subplot(1, 3, 2)
plt.plot(recall_xgb, precision_xgb, label=f'Average Precision = {avg_precision_xgb:.3f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")

# Calibration
plt.subplot(1, 3, 3)
plt.plot(prob_pred, prob_true, marker='o', linestyle='-',
         label='Calibration')
plt.plot([0, 1], [0, 1], 'k--', lw=0.8, label='Ideal')

plt.xlabel('Predicted probability')
plt.ylabel('Observed proportion')
plt.title('Calibration curve')
plt.text(0.05, 0.85,
         f'Intercept = {calib_intercept:.2f}\nSlope = {calib_slope:.2f}',
         transform=plt.gca().transAxes,
         bbox=dict(boxstyle='round', alpha=0.1))
plt.legend(loc='lower right')

plt.tight_layout()
plt.show()

## EBM Raw

In [ ]:
from interpret.glassbox import ExplainableBoostingClassifier

# Initialize the EBM model
ebm = ExplainableBoostingClassifier(random_state=42)

# Perform k-fold cross-validation
roc_auc_scores_ebm = cross_val_score(ebm, X_train_raw, y_train_raw, cv=kfold, scoring='roc_auc')
avg_precision_scores_ebm = cross_val_score(ebm, X_train_raw, y_train_raw, cv=kfold, scoring='average_precision')

# Train the model on the entire training set
ebm.fit(X_train_raw, y_train_raw)

# Predict on the test set
y_pred_prob_ebm = ebm.predict_proba(X_test_validated)[:, 1]

# Evaluate the model
roc_auc_ebm = roc_auc_score(y_test_validated, y_pred_prob_ebm)
avg_precision_ebm = average_precision_score(y_test_validated, y_pred_prob_ebm)

print(f"ROC AUC (EBM): {roc_auc_ebm:.3f}")
print(f"Average Precision (EBM): {avg_precision_ebm:.3f}")
print(f"Cross-validated ROC AUC scores (EBM): {roc_auc_scores_ebm}")
print(f"Cross-validated Average Precision scores (EBM): {avg_precision_scores_ebm}")

# Plot ROC and Precision-Recall curves
fpr_ebm, tpr_ebm, _ = roc_curve(y_test_validated, y_pred_prob_ebm)
precision_ebm, recall_ebm, _ = precision_recall_curve(y_test_validated, y_pred_prob_ebm)

# ─── Calibration curve (non-parametric bins) ────────────────────────────────
prob_true, prob_pred = calibration_curve(
    y_test_validated,
    y_pred_prob_ebm,
    n_bins=10,
    strategy='uniform'
)


# 5b) fit logistic in log-odds space
calib = LogisticRegression(penalty=None, solver='lbfgs', max_iter=1000)
eps   = 1e-15
p_clip = np.clip(y_pred_prob_ebm, eps, 1-eps)
logit_y_score = np.log(p_clip / (1 - p_clip))
calib.fit(logit_y_score.reshape(-1, 1), y_test_validated)
calib_intercept = calib.intercept_[0]
calib_slope = calib.coef_[0][0]

print(f"Calibration intercept (logit scale): {calib_intercept:.3f}")
print(f"Calibration slope     (logit scale): {calib_slope:.3f}")

# O : E ratio
exp_pos = np.sum(y_pred_prob_ebm)
obs_pos = np.sum(y_test_validated)
oe_ratio = obs_pos/exp_pos if exp_pos>0 else np.nan
print(f"O:E ratio (obs/exp): {oe_ratio:.3f}")

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(fpr_ebm, tpr_ebm, label=f'ROC AUC = {roc_auc_ebm:.3f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")

plt.subplot(1, 2, 2)
plt.plot(recall_ebm, precision_ebm, label=f'Average Precision = {avg_precision_ebm:.3f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")

# Calibration
plt.subplot(1, 3, 3)
plt.plot(prob_pred, prob_true, marker='o', linestyle='-',
         label='Calibration')
plt.plot([0, 1], [0, 1], 'k--', lw=0.8, label='Ideal')

plt.xlabel('Predicted probability')
plt.ylabel('Observed proportion')
plt.title('Calibration curve')
plt.text(0.05, 0.85,
         f'Intercept = {calib_intercept:.2f}\nSlope = {calib_slope:.2f}',
         transform=plt.gca().transAxes,
         bbox=dict(boxstyle='round', alpha=0.1))
plt.legend(loc='lower right')

plt.tight_layout()
plt.show()


# Train on Val

In [ ]:
# Train on validated data
validated_mask = train_set['source'] == 'validated'
X_train_validated = train_set[validated_mask].drop(columns=['ID', 'source'])
y_train_validated = y[train_set[validated_mask].index]

# Initialize the logistic regression model
logistic_regression = LogisticRegression(max_iter=1000, random_state=42)

# Perform k-fold cross-validation
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
roc_auc_scores = cross_val_score(logistic_regression, X_train_validated, y_train_validated, cv=kfold, scoring='roc_auc')
avg_precision_scores = cross_val_score(logistic_regression, X_train_validated, y_train_validated, cv=kfold, scoring='average_precision')

# Train the model on the entire training set
logistic_regression.fit(X_train_validated, y_train_validated)

# Predict on the validated test set
y_pred_prob_validated = logistic_regression.predict_proba(X_test_validated)[:, 1]

# Evaluate the model
roc_auc_validated = roc_auc_score(y_test_validated, y_pred_prob_validated)
avg_precision_validated = average_precision_score(y_test_validated, y_pred_prob_validated)

print(f"ROC AUC (Validated Data): {roc_auc_validated:.3f}")
print(f"Average Precision (Validated Data): {avg_precision_validated:.3f}")
print(f"Cross-validated ROC AUC scores (Raw Data): {roc_auc_scores}")
print(f"Cross-validated Average Precision scores (Raw Data): {avg_precision_scores}")

# ─── Calibration curve ─────────────────────────────────────────────────────────
prob_true, prob_pred = calibration_curve(y_test_validated,
                                         y_pred_prob_validated,
                                         n_bins=10, strategy='uniform')

# ─── Calibration slope & intercept (Steyerberg approach) ───────────────────────
eps = 1e-15                                                     # avoid log(0)
logit_pred = np.log((y_pred_prob_validated + eps) /
                    (1 - y_pred_prob_validated + eps)).reshape(-1, 1)

cal_model = SKLogit(penalty=None, solver='lbfgs', max_iter=1000)
cal_model.fit(logit_pred, y_test_validated)

cal_intercept = cal_model.intercept_[0]
cal_slope     = cal_model.coef_[0][0]

print(f"Calibration intercept: {cal_intercept:.3f}")
print(f"Calibration slope:     {cal_slope:.3f}")

# Optional: Plot ROC and Precision-Recall curves
fpr, tpr, _ = roc_curve(y_test_validated, y_pred_prob_validated)
precision, recall, _ = precision_recall_curve(y_test_validated, y_pred_prob_validated)

plt.figure(figsize=(13, 7), dpi=300)

#plt.rcParams["font.family"] = "Times New Roman"

plt.subplot(1, 3, 1)
plt.plot(fpr, tpr, label=f'ROC AUC = {roc_auc_validated:.3f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=24)
plt.ylabel('True Positive Rate', fontsize=24)
plt.title('ROC Curve', fontsize=28)
plt.legend(loc="lower right", fontsize=20)

plt.subplot(1, 3, 2)
plt.plot(recall, precision, label=f'Average Precision = {avg_precision_validated:.3f}')
plt.xlabel('Recall', fontsize=24)
plt.ylabel('Precision', fontsize=24)
plt.title('Precision-Recall Curve', fontsize=28)
plt.legend(loc="lower left", fontsize=20)

# Calibration
plt.subplot(1, 3, 3)
plt.plot(prob_pred, prob_true, marker='o', linestyle='-', label='Calibration')
plt.plot([0, 1], [0, 1], 'k--', lw=0.8, label='Ideal')
plt.xlabel('Predicted probability'); plt.ylabel('Observed proportion')
plt.title('Calibration curve')
plt.text(0.05, 0.85,
         f'Intercept = {cal_intercept:.2f}\nSlope = {cal_slope:.2f}',
         transform=plt.gca().transAxes,
         bbox=dict(boxstyle='round', alpha=0.1))
plt.legend(loc='lower right')
#plt.savefig("val_roc_jbhi.png", dpi=300, bbox_inches='tight')

plt.tight_layout()
plt.show()

In [ ]:
val_gdm_prob = y_pred_prob_validated

In [ ]:
plt.figure(figsize=(13, 7), dpi=300)

#plt.rcParams["font.family"] = "Times New Roman"

plt.subplot(1, 2, 1)
plt.plot(fpr, tpr, label=f'ROC AUC = {roc_auc_validated:.3f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=24)
plt.ylabel('True Positive Rate', fontsize=24)
plt.title('ROC Curve', fontsize=28)
plt.legend(loc="lower right", fontsize=20)

plt.subplot(1, 2, 2)
plt.plot(recall, precision, label=f'Average Precision = {avg_precision_validated:.3f}')
plt.xlabel('Recall', fontsize=24)
plt.ylabel('Precision', fontsize=24)
plt.title('Precision-Recall Curve', fontsize=28)
plt.legend(loc="lower left", fontsize=20)

#plt.savefig("raw_roc_jbhi.png", dpi=300, bbox_inches='tight', transparent=True)

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(13, 7), dpi=300)
plt.plot(prob_pred, prob_true, marker='o', linestyle='-', label='Calibration')
plt.plot([0, 1], [0, 1], 'k--', lw=0.8, label='Ideal')
plt.xlabel('Predicted probability'); plt.ylabel('Observed proportion')
plt.title('Calibration curve')
plt.text(0.05, 0.85,
         f'Intercept = {cal_intercept:.2f}\nSlope = {cal_slope:.2f}',
         transform=plt.gca().transAxes,
         bbox=dict(boxstyle='round', alpha=0.1))
plt.legend(loc='lower right')

## RF Val

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the Random Forest model
random_forest = RandomForestClassifier(random_state=42)

# Perform k-fold cross-validation
roc_auc_scores_rf = cross_val_score(random_forest, X_train_validated, y_train_validated, cv=kfold, scoring='roc_auc')
avg_precision_scores_rf = cross_val_score(random_forest, X_train_validated, y_train_validated, cv=kfold, scoring='average_precision')

# Train the model on the entire training set
random_forest.fit(X_train_validated, y_train_validated)

# Predict on the test set
y_pred_prob_rf = random_forest.predict_proba(X_test_validated)[:, 1]

# Evaluate the model
roc_auc_rf = roc_auc_score(y_test_validated, y_pred_prob_rf)
avg_precision_rf = average_precision_score(y_test_validated, y_pred_prob_rf)

print(f"ROC AUC (Random Forest): {roc_auc_rf:.3f}")
print(f"Average Precision (Random Forest): {avg_precision_rf:.3f}")
print(f"Cross-validated ROC AUC scores (Random Forest): {roc_auc_scores_rf}")
print(f"Cross-validated Average Precision scores (Random Forest): {avg_precision_scores_rf}")


# ─── Calibration curve (non-parametric bins) ────────────────────────────────
prob_true, prob_pred = calibration_curve(
    y_test_validated,
    y_pred_prob_rf,
    n_bins=10,
    strategy='uniform'
)

# linear least-squares in prob space
lr = LinearRegression().fit(prob_pred.reshape(-1,1), prob_true)
b0, b1 = lr.intercept_, lr.coef_[0]
print(f"Linear calib intercept: {b0:.3f}")
print(f"Linear calib slope:     {b1:.3f}")

# 5b) fit logistic in log-odds space
calib = LogisticRegression(penalty=None, solver='lbfgs', max_iter=1000)
eps   = 1e-15
p_clip = np.clip(y_pred_prob_rf, eps, 1-eps)
logodds = logit(p_clip).reshape(-1,1)
calib.fit(logodds, y_test_validated)

α = calib.intercept_[0]
β = calib.coef_[0,0]
print(f"Calibration intercept (logit scale): {α:.3f}")
print(f"Calibration slope     (logit scale): {β:.3f}")

# O : E ratio
exp_pos = np.sum(y_pred_prob_rf)
obs_pos = np.sum(y_test_validated)
oe_ratio = obs_pos/exp_pos if exp_pos>0 else np.nan
print(f"O:E ratio (obs/exp): {oe_ratio:.3f}")

# Plot ROC and Precision-Recall curves
fpr_rf, tpr_rf, _ = roc_curve(y_test_validated, y_pred_prob_rf)
precision_rf, recall_rf, _ = precision_recall_curve(y_test_validated, y_pred_prob_rf)

plt.figure(figsize=(10, 5))

plt.subplot(1, 3, 1)
plt.plot(fpr_rf, tpr_rf, label=f'ROC AUC = {roc_auc_rf:.3f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")

plt.subplot(1, 3, 2)
plt.plot(recall_rf, precision_rf, label=f'Average Precision = {avg_precision_rf:.3f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")

# Calibration
plt.subplot(1, 3, 3)
plt.plot(prob_pred, prob_true, marker='o', linestyle='-',
         label='Calibration')
plt.plot([0, 1], [0, 1], 'k--', lw=0.8, label='Ideal')

plt.xlabel('Predicted probability')
plt.ylabel('Observed proportion')
plt.title('Calibration curve')
plt.text(0.05, 0.85,
         f'Intercept = {α:.2f}\nSlope = {β:.2f}',
         transform=plt.gca().transAxes,
         bbox=dict(boxstyle='round', alpha=0.1))
plt.legend(loc='lower right')

plt.tight_layout()
plt.show()

## XGB Val

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve, precision_recall_curve
import matplotlib.pyplot as plt

# Convert all columns in X_train_raw from object to float
X_train_validated = X_train_validated.astype(float)

xgboost = XGBClassifier(eval_metric='logloss', random_state=42)

# Perform k-fold cross-validation
roc_auc_scores_xgb = cross_val_score(xgboost, X_train_validated, y_train_validated, cv=kfold, scoring='roc_auc')
avg_precision_scores_xgb = cross_val_score(xgboost, X_train_validated, y_train_validated, cv=kfold, scoring='average_precision')

# Train the model on the entire training set
xgboost.fit(X_train_validated, y_train_validated)

# Predict on the test set
y_pred_prob_xgb = xgboost.predict_proba(X_test_validated)[:, 1]

# Evaluate the model
roc_auc_xgb = roc_auc_score(y_test_validated, y_pred_prob_xgb)
avg_precision_xgb = average_precision_score(y_test_validated, y_pred_prob_xgb)

print(f"ROC AUC (XGBoost): {roc_auc_xgb:.3f}")
print(f"Average Precision (XGBoost): {avg_precision_xgb:.3f}")
print(f"Cross-validated ROC AUC scores (XGBoost): {roc_auc_scores_xgb}")
print(f"Cross-validated Average Precision scores (XGBoost): {avg_precision_scores_xgb}")

# Plot ROC and Precision-Recall curves
fpr_xgb, tpr_xgb, _ = roc_curve(y_test_validated, y_pred_prob_xgb)
precision_xgb, recall_xgb, _ = precision_recall_curve(y_test_validated, y_pred_prob_xgb)

# ─── Calibration curve (non-parametric bins) ────────────────────────────────
prob_true, prob_pred = calibration_curve(
    y_test_validated,
    y_pred_prob_xgb,
    n_bins=10,
    strategy='uniform'
)


# 5b) fit logistic in log-odds space
calib = LogisticRegression(penalty=None, solver='lbfgs', max_iter=1000)
eps   = 1e-15
p_clip = np.clip(y_pred_prob_xgb, eps, 1-eps)
logit_y_score = np.log(p_clip / (1 - p_clip))
calib.fit(logit_y_score.reshape(-1, 1), y_test_validated)
calib_intercept = calib.intercept_[0]
calib_slope = calib.coef_[0][0]

print(f"Calibration intercept (logit scale): {calib_intercept:.3f}")
print(f"Calibration slope     (logit scale): {calib_slope:.3f}")

# O : E ratio
exp_pos = np.sum(y_pred_prob_xgb)
obs_pos = np.sum(y_test_validated)
oe_ratio = obs_pos/exp_pos if exp_pos>0 else np.nan
print(f"O:E ratio (obs/exp): {oe_ratio:.3f}")


plt.figure(figsize=(10, 5))

plt.subplot(1, 3, 1)
plt.plot(fpr_xgb, tpr_xgb, label=f'ROC AUC = {roc_auc_xgb:.3f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")

plt.subplot(1, 3, 2)
plt.plot(recall_xgb, precision_xgb, label=f'Average Precision = {avg_precision_xgb:.3f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")

# Calibration
plt.subplot(1, 3, 3)
plt.plot(prob_pred, prob_true, marker='o', linestyle='-',
         label='Calibration')
plt.plot([0, 1], [0, 1], 'k--', lw=0.8, label='Ideal')

plt.xlabel('Predicted probability')
plt.ylabel('Observed proportion')
plt.title('Calibration curve')
plt.text(0.05, 0.85,
         f'Intercept = {calib_intercept:.2f}\nSlope = {calib_slope:.2f}',
         transform=plt.gca().transAxes,
         bbox=dict(boxstyle='round', alpha=0.1))
plt.legend(loc='lower right')

plt.tight_layout()
plt.show()

## EBM Val

In [ ]:
from interpret.glassbox import ExplainableBoostingClassifier

# Initialize the EBM model
ebm = ExplainableBoostingClassifier(random_state=42)

# Perform k-fold cross-validation
roc_auc_scores_ebm = cross_val_score(ebm, X_train_validated, y_train_validated, cv=kfold, scoring='roc_auc')
avg_precision_scores_ebm = cross_val_score(ebm, X_train_validated, y_train_validated, cv=kfold, scoring='average_precision')

# Train the model on the entire training set
ebm.fit(X_train_validated, y_train_validated)

# Predict on the test set
y_pred_prob_ebm = ebm.predict_proba(X_test_validated)[:, 1]

# Evaluate the model
roc_auc_ebm = roc_auc_score(y_test_validated, y_pred_prob_ebm)
avg_precision_ebm = average_precision_score(y_test_validated, y_pred_prob_ebm)

print(f"ROC AUC (EBM): {roc_auc_ebm:.3f}")
print(f"Average Precision (EBM): {avg_precision_ebm:.3f}")
print(f"Cross-validated ROC AUC scores (EBM): {roc_auc_scores_ebm}")
print(f"Cross-validated Average Precision scores (EBM): {avg_precision_scores_ebm}")

# Plot ROC and Precision-Recall curves
fpr_ebm, tpr_ebm, _ = roc_curve(y_test_validated, y_pred_prob_ebm)
precision_ebm, recall_ebm, _ = precision_recall_curve(y_test_validated, y_pred_prob_ebm)

# ─── Calibration curve (non-parametric bins) ────────────────────────────────
prob_true, prob_pred = calibration_curve(
    y_test_validated,
    y_pred_prob_ebm,
    n_bins=10,
    strategy='uniform'
)


# 5b) fit logistic in log-odds space
calib = LogisticRegression(penalty=None, solver='lbfgs', max_iter=1000)
eps   = 1e-15
p_clip = np.clip(y_pred_prob_ebm, eps, 1-eps)
logit_y_score = np.log(p_clip / (1 - p_clip))
calib.fit(logit_y_score.reshape(-1, 1), y_test_validated)
calib_intercept = calib.intercept_[0]
calib_slope = calib.coef_[0][0]

print(f"Calibration intercept (logit scale): {calib_intercept:.3f}")
print(f"Calibration slope     (logit scale): {calib_slope:.3f}")

# O : E ratio
exp_pos = np.sum(y_pred_prob_ebm)
obs_pos = np.sum(y_test_validated)
oe_ratio = obs_pos/exp_pos if exp_pos>0 else np.nan
print(f"O:E ratio (obs/exp): {oe_ratio:.3f}")

plt.figure(figsize=(10, 5))

plt.subplot(1, 3, 1)
plt.plot(fpr_ebm, tpr_ebm, label=f'ROC AUC = {roc_auc_ebm:.3f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")

plt.subplot(1, 3, 2)
plt.plot(recall_ebm, precision_ebm, label=f'Average Precision = {avg_precision_ebm:.3f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")

# Calibration
plt.subplot(1, 3, 3)
plt.plot(prob_pred, prob_true, marker='o', linestyle='-',
         label='Calibration')
plt.plot([0, 1], [0, 1], 'k--', lw=0.8, label='Ideal')

plt.xlabel('Predicted probability')
plt.ylabel('Observed proportion')
plt.title('Calibration curve')
plt.text(0.05, 0.85,
         f'Intercept = {calib_intercept:.2f}\nSlope = {calib_slope:.2f}',
         transform=plt.gca().transAxes,
         bbox=dict(boxstyle='round', alpha=0.1))
plt.legend(loc='lower right')

plt.tight_layout()
plt.show()


## Figures

### ROC & AP

In [ ]:
import matplotlib as mpl
# Compute ROC metrics
fpr_ehr, tpr_ehr, _ = roc_curve(val_gdm_true, ehr_gdm_prob)
fpr_val, tpr_val, _ = roc_curve(val_gdm_true, val_gdm_prob)
auc_ehr = roc_auc_score(val_gdm_true, ehr_gdm_prob)
auc_val = roc_auc_score(val_gdm_true, val_gdm_prob)

# Compute PR metrics
prec_ehr, rec_ehr, _ = precision_recall_curve(val_gdm_true, ehr_gdm_prob)
prec_val, rec_val, _ = precision_recall_curve(val_gdm_true, val_gdm_prob)
ap_ehr = average_precision_score(val_gdm_true, ehr_gdm_prob)
ap_val = average_precision_score(val_gdm_true, val_gdm_prob)

mpl.rcParams['font.family'] = 'Times New Roman'
mpl.rcParams['font.size']   = 16

# Plot
fig, (ax_roc, ax_pr) = plt.subplots(1, 2, figsize=(12, 5), dpi=300)

# ROC plot
ax_roc.plot(fpr_ehr, tpr_ehr, label=f'EHR-GDM (AUC = {auc_ehr:.3f})')
ax_roc.plot(fpr_val, tpr_val, label=f'VAL-GDM (AUC = {auc_val:.3f})')
ax_roc.plot([0,1], [0,1], '--', color='grey')
ax_roc.set_xlabel('False Positive Rate')
ax_roc.set_ylabel('True Positive Rate')
ax_roc.set_title('ROC Curves')
ax_roc.legend(loc='lower right', fontsize=14)

# PR plot
ax_pr.step(rec_ehr, prec_ehr, where='post', label=f'EHR-GDM (AP = {ap_ehr:.3f})')
ax_pr.step(rec_val, prec_val, where='post', label=f'VAL-GDM (AP = {ap_val:.3f})')
ax_pr.set_xlabel('Recall')
ax_pr.set_ylabel('Precision')
ax_pr.set_title('Precision–Recall Curves')
ax_pr.set_xlim(0, 1)
ax_pr.set_ylim(0, 1.05)
ax_pr.legend(loc='lower left', fontsize=14)

plt.tight_layout()
fig.savefig("roc_pr_comparison_jmir.png", dpi=300)
plt.show()

### Calibration Curve

In [ ]:
# Compute calibration for EHR-GDM
prob_true_ehr, prob_pred_ehr = calibration_curve(
    val_gdm_true,      # y_test_validated
    ehr_gdm_prob,      # y_pred_prob_validated for EHR
    n_bins=10,
    strategy='uniform'
)

# Compute calibration for VAL-GDM
prob_true_val, prob_pred_val = calibration_curve(
    val_gdm_true,      # same ground truth
    val_gdm_prob,      # y_pred_prob_validated for VAL
    n_bins=10,
    strategy='uniform'
)
# Plot
fig, (ax_ehr, ax_val) = plt.subplots(1, 2, figsize=(12, 5), dpi=300)

# Left: EHR calibration
ax_ehr.plot(prob_pred_ehr, prob_true_ehr, marker='o', label='EHR-GDM')
ax_ehr.plot([0,1],[0,1],'--', color='grey')
ax_ehr.set_xlabel('Mean Predicted Probability')
ax_ehr.set_ylabel('Observed Proportion')
ax_ehr.set_title('Calibration Curve for EHR-GDM')
ax_ehr.legend(loc='lower right')

# Right: VAL calibration
ax_val.plot(prob_pred_val, prob_true_val, marker='o', label='VAL-GDM')
ax_val.plot([0,1],[0,1],'--', color='grey')
ax_val.set_xlabel('Mean Predicted Probability')
ax_val.set_ylabel('Observed Proportion')
ax_val.set_title('Calibration Curve for VAL-GDM')
ax_val.legend(loc='lower right')

plt.tight_layout()
fig.savefig("calibration_jmir.png", dpi=300)
plt.show()

# Label Noise Train

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score, confusion_matrix, classification_report, roc_curve, precision_recall_curve, auc
import matplotlib.pyplot as plt
import random

# Define categorical and numerical features
categorical_features = merged_df.select_dtypes(include=['object']).columns.tolist()
numerical_features = merged_df.select_dtypes(exclude=['object']).columns.tolist()

# Remove target and identifier columns from features
target_column = 'GDM'
numerical_features.remove(target_column)
numerical_features.remove('ID')

# Preprocessor setup
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), categorical_features),
        ('num', StandardScaler(), numerical_features)
    ],
    remainder='passthrough'
)

# Apply preprocessing to the entire dataset except 'GDM' and 'ID'
X = merged_df.drop(columns=[target_column, 'ID'])
X_processed = preprocessor.fit_transform(X)
y = merged_df[target_column].values

# Convert processed features back to DataFrame with transformed column names
columns_transformed = (list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features))
                       + numerical_features)
X_processed_df = pd.DataFrame(X_processed, columns=columns_transformed)
X_processed_df['ID'] = merged_df['ID'].values

# Initial group-based split to create train and test sets
gss = GroupShuffleSplit(n_splits=1, train_size=0.7, random_state=42)
train_idxs, test_idxs = next(gss.split(X_processed_df, groups=X_processed_df['ID']))

train_set = X_processed_df.iloc[train_idxs]
test_set = X_processed_df.iloc[test_idxs]
y_train = y[train_idxs]
y_test = y[test_idxs]

# Separate features from 'ID' for training and test sets
X_train = train_set.drop(columns=['ID'])
X_test = test_set.drop(columns=['ID'])

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score
import matplotlib.pyplot as plt

# Function to Add Incremental Label Noise, ensuring only original labels are flipped
def add_incremental_label_noise(y, pos_noise_level, neg_noise_level):
    """
    Add incremental label noise to a binary target array, ensuring that only original labels are flipped.
    
    Parameters:
    y (np.array): The original labels.
    pos_noise_level (float): The proportion of positive labels (1s) to be flipped to 0s.
    neg_noise_level (float): The proportion of negative labels (0s) to be flipped to 1s.
    
    Returns:
    np.array: The labels with noise introduced.
    """
    # Make a copy of the original labels
    y_noisy = y.copy()
    
    # Identify original positive and negative indices
    original_pos_indices = np.where(y == 1)[0]
    original_neg_indices = np.where(y == 0)[0]
    
    # Calculate the number of labels to flip
    n_pos_noisy = int(pos_noise_level * len(original_pos_indices))
    n_neg_noisy = int(neg_noise_level * len(original_neg_indices))
    
    # Select indices to flip
    pos_noisy_indices = np.random.choice(original_pos_indices, n_pos_noisy, replace=False)
    neg_noisy_indices = np.random.choice(original_neg_indices, n_neg_noisy, replace=False)
    
    # Flip the selected original labels only
    y_noisy[pos_noisy_indices] = 0
    y_noisy[neg_noisy_indices] = 1
    
    return y_noisy


# Initialize the logistic regression model
logistic_regression = LogisticRegression(max_iter=1000, random_state=42)

# Different levels of noise to evaluate
noise_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]  # Example noise levels

# Evaluate model performance with different noise levels
performance_metrics_incremental = []

# Evaluate model performance with incremental label noise
for pos_noise_level in noise_levels:
    for neg_noise_level in noise_levels:
        print(f"Evaluating pos_noise_level: {pos_noise_level}, neg_noise_level: {neg_noise_level}")
        
        # Add incremental label noise to the training data
        y_train_noisy = add_incremental_label_noise(y_train, pos_noise_level, neg_noise_level)

        # Train the model on noisy training data
        logistic_regression.fit(X_train, y_train_noisy)

        # Predict on the clean test set
        y_pred_proba = logistic_regression.predict_proba(X_test)[:, 1]

        # Evaluate performance on the clean test set
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        ap_score = average_precision_score(y_test, y_pred_proba)

        # Store metrics as dictionaries
        metrics = {
            'pos_noise_level': pos_noise_level,
            'neg_noise_level': neg_noise_level,
            'roc_auc_mean': roc_auc,
            'average_precision_mean': ap_score,
        }
        performance_metrics_incremental.append(metrics)

# Display AUC and AP for each noise level combination
for metrics in performance_metrics_incremental:
    pos_noise_level_train = metrics['pos_noise_level']
    neg_noise_level_train = metrics['neg_noise_level']
    roc_auc_mean_train = metrics['roc_auc_mean']
    average_precision_mean_train = metrics['average_precision_mean']
    
    print(f"Pos Noise Level: {pos_noise_level_train*100:.0f}%, Neg Noise Level: {neg_noise_level_train*100:.0f}%")
    print(f"  ROC AUC Mean: {roc_auc_mean_train:.3f}")
    print(f"  Average Precision Mean: {average_precision_mean_train:.3f}")
    print("\n")

# Plot ROC AUC and Average Precision for different noise levels with incremental label noise
# Extract values for plotting
pos_noise_levels_train = sorted(list(set([metrics['pos_noise_level'] for metrics in performance_metrics_incremental])))
neg_noise_levels_train = sorted(list(set([metrics['neg_noise_level'] for metrics in performance_metrics_incremental])))

roc_auc_matrix_train = np.zeros((len(pos_noise_levels_train), len(neg_noise_levels_train)))
ap_matrix_train = np.zeros((len(pos_noise_levels_train), len(neg_noise_levels_train)))

for metrics in performance_metrics_incremental:
    pos_index_train = pos_noise_levels_train.index(metrics['pos_noise_level'])
    neg_index_train = neg_noise_levels_train.index(metrics['neg_noise_level'])
    roc_auc_matrix_train[pos_index_train, neg_index_train] = metrics['roc_auc_mean']
    ap_matrix_train[pos_index_train, neg_index_train] = metrics['average_precision_mean']

# Plot ROC AUC
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.imshow(roc_auc_matrix_train, interpolation='nearest', cmap='viridis')
plt.title('ROC AUC vs Noise Levels')
plt.xlabel('Neg Noise Level')
plt.ylabel('Pos Noise Level')
plt.colorbar(label='ROC AUC')
plt.xticks(ticks=np.arange(len(neg_noise_levels_train)), labels=[f'{nl*100:.0f}%' for nl in neg_noise_levels_train])
plt.yticks(ticks=np.arange(len(pos_noise_levels_train)), labels=[f'{pl*100:.0f}%' for pl in pos_noise_levels_train])

# Annotate ROC AUC values
for i in range(len(pos_noise_levels_train)):
    for j in range(len(neg_noise_levels_train)):
        plt.text(j, i, f'{roc_auc_matrix_train[i, j]:.2f}', ha='center', va='center', color='white')

# Plot Average Precision
plt.subplot(1, 2, 2)
plt.imshow(ap_matrix_train, interpolation='nearest', cmap='viridis')
plt.title('Average Precision vs Noise Levels')
plt.xlabel('Neg Noise Level')
plt.ylabel('Pos Noise Level')
plt.colorbar(label='Average Precision')
plt.xticks(ticks=np.arange(len(neg_noise_levels_train)), labels=[f'{nl*100:.0f}%' for nl in neg_noise_levels_train])
plt.yticks(ticks=np.arange(len(pos_noise_levels_train)), labels=[f'{pl*100:.0f}%' for pl in pos_noise_levels_train])

# Annotate AP values
for i in range(len(pos_noise_levels_train)):
    for j in range(len(neg_noise_levels_train)):
        plt.text(j, i, f'{ap_matrix_train[i, j]:.2f}', ha='center', va='center', color='white')

plt.tight_layout()
plt.show()

In [ ]:
# Plot ROC AUC
plt.figure(figsize=(14, 6), dpi=300)

plt.subplot(1, 2, 1)
plt.imshow(roc_auc_matrix_train, interpolation='nearest', cmap='cividis')
plt.title('Increasing Noise in Train Set: \nROC AUC', fontsize=20)
plt.xlabel('Increasing False Positives', fontsize=18)
plt.ylabel('Increasing False Negatives', fontsize=18)
plt.colorbar(label='ROC AUC')
plt.xticks(ticks=np.arange(len(neg_noise_levels_train)), labels=[f'{nl*100:.0f}%' for nl in neg_noise_levels_train], fontsize=12)
plt.yticks(ticks=np.arange(len(pos_noise_levels_train)), labels=[f'{pl*100:.0f}%' for pl in pos_noise_levels_train], fontsize=12)

# Annotate ROC AUC values with contrasting color based on background intensity
for i in range(len(pos_noise_levels_train)):
    for j in range(len(neg_noise_levels_train)):
        value = roc_auc_matrix_train[i, j]
        color = 'midnightblue' if value > 0.7 else 'white'  # Choose color based on a threshold
        plt.text(j, i, f'{value:.2f}', ha='center', va='center', color=color, fontsize=10)

# Plot Average Precision
plt.subplot(1, 2, 2)
plt.imshow(ap_matrix_train, interpolation='nearest', cmap='cividis')
plt.title('Increasing Noise in Train Set: \nAverage Precision', fontsize=20)
plt.xlabel('Increasing False Positives', fontsize=18)
plt.ylabel('Increasing False Negatives', fontsize=18)
plt.colorbar(label='Average Precision')
plt.xticks(ticks=np.arange(len(neg_noise_levels_train)), labels=[f'{nl*100:.0f}%' for nl in neg_noise_levels_train], fontsize=12)
plt.yticks(ticks=np.arange(len(pos_noise_levels_train)), labels=[f'{pl*100:.0f}%' for pl in pos_noise_levels_train], fontsize=12)

# Annotate AP values with contrasting color based on background intensity
for i in range(len(pos_noise_levels_train)):
    for j in range(len(neg_noise_levels_train)):
        value = ap_matrix_train[i, j]
        color = 'midnightblue' if value > 0.4 else 'white'  # Choose color based on threshold
        plt.text(j, i, f'{value:.2f}', ha='center', va='center', color=color, fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# Plot ROC AUC
plt.figure(figsize=(14, 6), dpi=500)

plt.subplot(1, 2, 1)
plt.imshow(roc_auc_matrix, interpolation='nearest', cmap='cividis')
plt.title('ROC AUC vs Noise Levels', fontsize=20)
plt.xlabel('Neg Noise Level', fontsize=18)
plt.ylabel('Pos Noise Level', fontsize=18)
plt.colorbar(label='ROC AUC')
plt.xticks(ticks=np.arange(len(neg_noise_levels)), labels=[f'{nl*100:.0f}%' for nl in neg_noise_levels], fontsize=12)
plt.yticks(ticks=np.arange(len(pos_noise_levels)), labels=[f'{pl*100:.0f}%' for pl in pos_noise_levels], fontsize=12)

# Annotate ROC AUC values with contrasting color based on background intensity
for i in range(len(pos_noise_levels)):
    for j in range(len(neg_noise_levels)):
        value = roc_auc_matrix[i, j]
        color = 'midnightblue' if value > 0.7 else 'white'  # Choose color based on a threshold
        plt.text(j, i, f'{value:.2f}', ha='center', va='center', color=color, fontsize=10)

# Plot Average Precision
plt.subplot(1, 2, 2)
plt.imshow(ap_matrix, interpolation='nearest', cmap='cividis')
plt.title('Average Precision vs Noise Levels', fontsize=20)
plt.xlabel('Neg Noise Level', fontsize=18)
plt.ylabel('Pos Noise Level', fontsize=18)
plt.colorbar(label='Average Precision')
plt.xticks(ticks=np.arange(len(neg_noise_levels)), labels=[f'{nl*100:.0f}%' for nl in neg_noise_levels], fontsize=12)
plt.yticks(ticks=np.arange(len(pos_noise_levels)), labels=[f'{pl*100:.0f}%' for pl in pos_noise_levels], fontsize=12)

# Annotate AP values with contrasting color based on background intensity
for i in range(len(pos_noise_levels)):
    for j in range(len(neg_noise_levels)):
        value = ap_matrix[i, j]
        color = 'midnightblue' if value > 0.4 else 'white'  # Choose color based on threshold
        plt.text(j, i, f'{value:.2f}', ha='center', va='center', color=color, fontsize=10)

plt.tight_layout()
plt.show()

# Label Noise Test

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score
import matplotlib.pyplot as plt

# Function to Add Incremental Label Noise, ensuring only original labels are flipped
def add_incremental_label_noise(y, pos_noise_level, neg_noise_level):
    """
    Add incremental label noise to a binary target array, ensuring that only original labels are flipped.
    
    Parameters:
    y (np.array): The original labels.
    pos_noise_level (float): The proportion of positive labels (1s) to be flipped to 0s.
    neg_noise_level (float): The proportion of negative labels (0s) to be flipped to 1s.
    
    Returns:
    np.array: The labels with noise introduced.
    """
    # Make a copy of the original labels
    y_noisy = y.copy()
    
    # Identify original positive and negative indices
    original_pos_indices = np.where(y == 1)[0]
    original_neg_indices = np.where(y == 0)[0]
    
    # Calculate the number of labels to flip
    n_pos_noisy = int(pos_noise_level * len(original_pos_indices))
    n_neg_noisy = int(neg_noise_level * len(original_neg_indices))
    
    # Select indices to flip
    pos_noisy_indices = np.random.choice(original_pos_indices, n_pos_noisy, replace=False)
    neg_noisy_indices = np.random.choice(original_neg_indices, n_neg_noisy, replace=False)
    
    # Flip the selected original labels only
    y_noisy[pos_noisy_indices] = 0
    y_noisy[neg_noisy_indices] = 1
    
    return y_noisy


# Initialize the logistic regression model
logistic_regression = LogisticRegression(max_iter=1000, random_state=42)

# Different levels of noise to evaluate
noise_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]  # Example noise levels


# Evaluate model performance with different noise levels
performance_metrics_incremental = []

# Evaluate model performance with incremental label noise
for pos_noise_level in noise_levels:
    for neg_noise_level in noise_levels:
        print(f"Evaluating pos_noise_level: {pos_noise_level}, neg_noise_level: {neg_noise_level}")
        
        # Add incremental label noise to the training data
        y_test_noisy = add_incremental_label_noise(y_test, pos_noise_level, neg_noise_level)

        # Train the model
        logistic_regression.fit(X_train, y_train)

        # Predict on the validation set
        y_pred_proba = logistic_regression.predict_proba(X_test)[:, 1]

        # Evaluate performance
        roc_auc = roc_auc_score(y_test_noisy, y_pred_proba)
        ap_score = average_precision_score(y_test_noisy, y_pred_proba)

        # Store metrics as dictionaries
        metrics = {
            'pos_noise_level': pos_noise_level,
            'neg_noise_level': neg_noise_level,
            'roc_auc_mean': roc_auc,
            'average_precision_mean': ap_score,
        }
        performance_metrics_incremental.append(metrics)

# Display AUC and AP for each noise level combination
for metrics in performance_metrics_incremental:
    pos_noise_level = metrics['pos_noise_level']
    neg_noise_level = metrics['neg_noise_level']
    roc_auc_mean = metrics['roc_auc_mean']
    average_precision_mean = metrics['average_precision_mean']
    
    print(f"Pos Noise Level: {pos_noise_level*100:.0f}%, Neg Noise Level: {neg_noise_level*100:.0f}%")
    print(f"  ROC AUC Mean: {roc_auc_mean:.3f}")
    print(f"  Average Precision Mean: {average_precision_mean:.3f}")
    print("\n")

# Plot ROC AUC and Average Precision for different noise levels with incremental label noise
# Extract values for plotting
pos_noise_levels = sorted(list(set([metrics['pos_noise_level'] for metrics in performance_metrics_incremental])))
neg_noise_levels = sorted(list(set([metrics['neg_noise_level'] for metrics in performance_metrics_incremental])))

roc_auc_matrix = np.zeros((len(pos_noise_levels), len(neg_noise_levels)))
ap_matrix = np.zeros((len(pos_noise_levels), len(neg_noise_levels)))

for metrics in performance_metrics_incremental:
    pos_index = pos_noise_levels.index(metrics['pos_noise_level'])
    neg_index = neg_noise_levels.index(metrics['neg_noise_level'])
    roc_auc_matrix[pos_index, neg_index] = metrics['roc_auc_mean']
    ap_matrix[pos_index, neg_index] = metrics['average_precision_mean']

# Plot ROC AUC
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.imshow(roc_auc_matrix, interpolation='nearest', cmap='viridis')
plt.title('ROC AUC vs Noise Levels')
plt.xlabel('Neg Noise Level')
plt.ylabel('Pos Noise Level')
plt.colorbar(label='ROC AUC')
plt.xticks(ticks=np.arange(len(neg_noise_levels)), labels=[f'{nl*100:.0f}%' for nl in neg_noise_levels])
plt.yticks(ticks=np.arange(len(pos_noise_levels)), labels=[f'{pl*100:.0f}%' for pl in pos_noise_levels])

# Annotate ROC AUC values
for i in range(len(pos_noise_levels)):
    for j in range(len(neg_noise_levels)):
        plt.text(j, i, f'{roc_auc_matrix[i, j]:.2f}', ha='center', va='center', color='white')

# Plot Average Precision
plt.subplot(1, 2, 2)
plt.imshow(ap_matrix, interpolation='nearest', cmap='viridis')
plt.title('Average Precision vs Noise Levels')
plt.xlabel('Neg Noise Level')
plt.ylabel('Pos Noise Level')
plt.colorbar(label='Average Precision')
plt.xticks(ticks=np.arange(len(neg_noise_levels)), labels=[f'{nl*100:.0f}%' for nl in neg_noise_levels])
plt.yticks(ticks=np.arange(len(pos_noise_levels)), labels=[f'{pl*100:.0f}%' for pl in pos_noise_levels])

# Annotate AP values
for i in range(len(pos_noise_levels)):
    for j in range(len(neg_noise_levels)):
        plt.text(j, i, f'{ap_matrix[i, j]:.2f}', ha='center', va='center', color='white')

plt.tight_layout()
plt.show()

In [ ]:
# Plot ROC AUC
plt.figure(figsize=(14, 6), dpi=300)

plt.subplot(1, 2, 1)
plt.imshow(roc_auc_matrix, interpolation='nearest', cmap='viridis')
plt.title('Increasing Noise in Test Set: \nROC AUC', fontsize=20)
plt.xlabel('Increasing False Positives', fontsize=18)
plt.ylabel('Increasing False Negatives', fontsize=18)
plt.colorbar(label='ROC AUC')
plt.xticks(ticks=np.arange(len(neg_noise_levels)), labels=[f'{nl*100:.0f}%' for nl in neg_noise_levels], fontsize=12)
plt.yticks(ticks=np.arange(len(pos_noise_levels)), labels=[f'{pl*100:.0f}%' for pl in pos_noise_levels], fontsize=12)

# Annotate ROC AUC values with contrasting color based on background intensity
for i in range(len(pos_noise_levels)):
    for j in range(len(neg_noise_levels)):
        value = roc_auc_matrix[i, j]
        color = 'midnightblue' if value > 0.7 else 'white'  # Choose color based on a threshold
        plt.text(j, i, f'{value:.2f}', ha='center', va='center', color=color, fontsize=10)

# Plot Average Precision
plt.subplot(1, 2, 2)
plt.imshow(ap_matrix, interpolation='nearest', cmap='viridis')
plt.title('Increasing Noise in Test Set: \nAverage Precision', fontsize=20)
plt.xlabel('Increasing False Positives', fontsize=18)
plt.ylabel('Increasing False Negatives', fontsize=18)
plt.colorbar(label='Average Precision')
plt.xticks(ticks=np.arange(len(neg_noise_levels)), labels=[f'{nl*100:.0f}%' for nl in neg_noise_levels], fontsize=12)
plt.yticks(ticks=np.arange(len(pos_noise_levels)), labels=[f'{pl*100:.0f}%' for pl in pos_noise_levels],fontsize=12)

# Annotate AP values with contrasting color based on background intensity
for i in range(len(pos_noise_levels)):
    for j in range(len(neg_noise_levels)):
        value = ap_matrix[i, j]
        color = 'midnightblue' if value > 0.7 else 'white'  # Choose color based on threshold
        plt.text(j, i, f'{value:.2f}', ha='center', va='center', color=color, fontsize=10)

plt.tight_layout()
plt.show()

## Figures

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(14, 12), dpi=300)

# Top-left: Train ROC AUC (cividis)
ax = axes[0,0]
im = ax.imshow(roc_auc_matrix_train, interpolation='nearest', cmap='cividis')
ax.set_title('Train Set – ROC AUC', fontsize=20)
ax.set_xlabel('False Positive Noise', fontsize=18)
ax.set_ylabel('False Negative Noise', fontsize=18)
ax.set_xticks(np.arange(len(neg_noise_levels_train)))
ax.set_xticklabels([f'{nl*100:.0f}%' for nl in neg_noise_levels_train], fontsize=12)
ax.set_yticks(np.arange(len(pos_noise_levels_train)))
ax.set_yticklabels([f'{pl*100:.0f}%' for pl in pos_noise_levels_train], fontsize=12)
cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label('ROC AUC', fontsize=14)

for i in range(len(pos_noise_levels_train)):
    for j in range(len(neg_noise_levels_train)):
        val = roc_auc_matrix_train[i, j]
        color = 'midnightblue' if val > 0.7 else 'white'
        ax.text(j, i, f'{val:.2f}', ha='center', va='center', color=color, fontsize=11)

# Top-right: Train AP (cividis)
ax = axes[0,1]
im = ax.imshow(ap_matrix_train, interpolation='nearest', cmap='cividis')
ax.set_title('Train Set – Average Precision', fontsize=20)
ax.set_xlabel('False Positive Noise', fontsize=18)
ax.set_ylabel('False Negative Noise', fontsize=18)
ax.set_xticks(np.arange(len(neg_noise_levels_train)))
ax.set_xticklabels([f'{nl*100:.0f}%' for nl in neg_noise_levels_train], fontsize=12)
ax.set_yticks(np.arange(len(pos_noise_levels_train)))
ax.set_yticklabels([f'{pl*100:.0f}%' for pl in pos_noise_levels_train], fontsize=12)
cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label('Average Precision', fontsize=14)

for i in range(len(pos_noise_levels_train)):
    for j in range(len(neg_noise_levels_train)):
        val = ap_matrix_train[i, j]
        color = 'midnightblue' if val > 0.4 else 'white'
        ax.text(j, i, f'{val:.2f}', ha='center', va='center', color=color, fontsize=11)

# Bottom-left: Test ROC AUC (viridis)
ax = axes[1,0]
im = ax.imshow(roc_auc_matrix, interpolation='nearest', cmap='viridis')
ax.set_title('Test Set – ROC AUC', fontsize=20)
ax.set_xlabel('False Positive Noise', fontsize=18)
ax.set_ylabel('False Negative Noise', fontsize=18)
ax.set_xticks(np.arange(len(neg_noise_levels)))
ax.set_xticklabels([f'{nl*100:.0f}%' for nl in neg_noise_levels], fontsize=12)
ax.set_yticks(np.arange(len(pos_noise_levels)))
ax.set_yticklabels([f'{pl*100:.0f}%' for pl in pos_noise_levels], fontsize=12)
cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label('ROC AUC', fontsize=14)

for i in range(len(pos_noise_levels)):
    for j in range(len(neg_noise_levels)):
        val = roc_auc_matrix[i, j]
        color = 'midnightblue' if val > 0.7 else 'white'
        ax.text(j, i, f'{val:.2f}', ha='center', va='center', color=color, fontsize=11)

# Bottom-right: Test AP (viridis)
ax = axes[1,1]
im = ax.imshow(ap_matrix, interpolation='nearest', cmap='viridis')
ax.set_title('Test Set – Average Precision', fontsize=20)
ax.set_xlabel('False Positive Noise', fontsize=18)
ax.set_ylabel('False Negative Noise', fontsize=18)
ax.set_xticks(np.arange(len(neg_noise_levels)))
ax.set_xticklabels([f'{nl*100:.0f}%' for nl in neg_noise_levels], fontsize=12)
ax.set_yticks(np.arange(len(pos_noise_levels)))
ax.set_yticklabels([f'{pl*100:.0f}%' for pl in pos_noise_levels], fontsize=12)
cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label('Average Precision', fontsize=14)

for i in range(len(pos_noise_levels)):
    for j in range(len(neg_noise_levels)):
        val = ap_matrix[i, j]
        color = 'midnightblue' if val > 0.7 else 'white'
        ax.text(j, i, f'{val:.2f}', ha='center', va='center', color=color, fontsize=11)

plt.tight_layout()
fig.savefig("label_noise.png", dpi=300)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

# Load your real matrices here:
# roc_auc_matrix_train, ap_matrix_train, roc_auc_matrix, ap_matrix
# pos_noise_levels_train, neg_noise_levels_train
# pos_noise_levels, neg_noise_levels

# Configure font
mpl.rcParams['font.family'] = 'Times New Roman'
mpl.rcParams['font.size']   = 16

fig, axes = plt.subplots(2, 2, figsize=(14, 12), dpi=300)

def plot_heatmap(ax, matrix, title, cmap, pos_levels, neg_levels, val_threshold, summary_text):
    im = ax.imshow(matrix, interpolation='nearest', cmap=cmap)
    ax.set_title(title, fontsize=20)
    ax.set_xlabel('False Positive Noise', fontsize=18)
    ax.set_ylabel('False Negative Noise', fontsize=18)
    ax.set_xticks(np.arange(len(neg_levels)))
    ax.set_xticklabels([f'{nl*100:.0f}%' for nl in neg_levels], fontsize=12)
    ax.set_yticks(np.arange(len(pos_levels)))
    ax.set_yticklabels([f'{pl*100:.0f}%' for pl in pos_levels], fontsize=12)
    cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label(title.split('–')[-1].strip(), fontsize=14)

    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            val = matrix[i, j]
            color = 'midnightblue' if val > val_threshold else 'white'
            ax.text(j, i, f'{val:.2f}', ha='center', va='center', color=color, fontsize=11)

    # Centered callout at top
    ax.text(0.5, 0.98, summary_text, transform=ax.transAxes,
            fontsize=12, va='top', ha='center',
            bbox=dict(boxstyle="round,pad=0.4", fc="white", ec="black", alpha=0.8))

# Plot each subplot with centered callout
plot_heatmap(
    axes[0,0],
    roc_auc_matrix_train,
    'Train Set – ROC AUC',
    cmap='cividis',
    pos_levels=pos_noise_levels_train,
    neg_levels=neg_noise_levels_train,
    val_threshold=0.7,
    summary_text="Lighter = better discrimination\nAUC > 0.5 better-than-chance performance"
)

plot_heatmap(
    axes[0,1],
    ap_matrix_train,
    'Train Set – Average Precision',
    cmap='cividis',
    pos_levels=pos_noise_levels_train,
    neg_levels=neg_noise_levels_train,
    val_threshold=0.4,
    summary_text="Lighter = better precision-recall"
)

plot_heatmap(
    axes[1,0],
    roc_auc_matrix,
    'Test Set – ROC AUC',
    cmap='viridis',
    pos_levels=pos_noise_levels,
    neg_levels=neg_noise_levels,
    val_threshold=0.7,
    summary_text="Lighter = better discrimination\nAUC > 0.5 better-than-chance performance"
)

plot_heatmap(
    axes[1,1],
    ap_matrix,
    'Test Set – Average Precision',
    cmap='viridis',
    pos_levels=pos_noise_levels,
    neg_levels=neg_noise_levels,
    val_threshold=0.6,
    summary_text="Lighter = better precision-recall"
)

plt.tight_layout()
fig.savefig("label_noise_callout.png", dpi=300)
plt.show()